# Dédupliquer à partir d'un excel

A partir du fichier excel de cluster dédupliqué validé par Christian, nous créons un acteur chapeau et nous ratachons les acteurs du cluster à ce chapeau

## Déclaration des librairies et des variables

In [161]:
import requests
import uuid

import pandas as pd
from shapely import wkb
from shapely.geometry import Point
from sqlalchemy import create_engine


DATABASE_URL = 'postgresql+psycopg2://…'
INTERFACE_HOST = "http://localhost:8000"
# Chemin vers le fichier Excel
FILE_PATH = '../cluster dechetteries - check_similarity_downstream_without_filtering_by_threshold.xlsx'


## Connection à la base de données

In [162]:
# Create the engine
engine = create_engine(DATABASE_URL)

## Récupération des id des sources ALIAPUR et COREPILE

Car on sait par expérience que ces 2 Eco-organismes ont des données plus propre, on préfèrera donc utiliser leur données plutôt que celle des autres Eco-organisme en cas de choix à faire, champ par champ pour créer l'acteur chapeau


In [164]:
# récupération des id des sources aliapur et corepile
query_acteur = "SELECT id, code FROM qfdmo_source WHERE code IN ('ALIAPUR','COREPILE');"
sources = pd.read_sql_query(query_acteur, engine)
trusted_sources = ['ALIAPUR','COREPILE']
trusted_source_ids = sources[sources['code'].isin(trusted_sources)]['id'].tolist()

# Ajouter l'id 0 au début de la liste trusted_source_ids pour doner la priorité
# aux acteurs chapeau déjà existant
trusted_source_ids.insert(0, 0)


## Gestion du fichier de déduplication

On récupère le fichier à dédupliquer localement et on le transforme en dataframe pandas

In [165]:
# Lire le fichier Excel et le convertir en DataFrame
df = pd.read_excel(FILE_PATH)

# On ne récupère ques les clusters d'identifiant_unique, le reste est retrouvé via
# la base de données
df_clusters = df[['cluster_id', 'identifiant_unique']]

## Récupération des acteurs et revisionacteurs de la base de données

In [166]:
# filtré par identifiant_unique
identifiant_unique = df_clusters['identifiant_unique'].tolist()
identifiant_unique_list = [f"'{i}'" for i in identifiant_unique]

query_acteur = f"SELECT * FROM qfdmo_acteur WHERE identifiant_unique IN ({','.join(identifiant_unique_list)});"
df_acteur_from_sql = pd.read_sql_query(query_acteur, engine)

query_revisionacteur = f"SELECT * FROM qfdmo_revisionacteur WHERE identifiant_unique IN ({','.join(identifiant_unique_list)});"
df_revisionacteur_from_sql = pd.read_sql_query(query_revisionacteur, engine)

# merge acteur et revisionacteur
df_acteur_from_sql['parent_id'] = None
df_acteur_from_sql.set_index("identifiant_unique", inplace=True)
df_revisionacteur_from_sql.set_index("identifiant_unique", inplace=True)

# mise à jour avec les données de revisionacteur si elles existent
df_acteur_from_sql.update(df_revisionacteur_from_sql)
df_acteur_from_sql.reset_index(inplace=True)
df_acteur_from_sql['is_parent'] = False


In [167]:
# récupération des parents
df_parentacteur_from_sql = pd.DataFrame(
    columns=df_acteur_from_sql.columns
)
df_enfantacteur_from_sql = pd.DataFrame(
    columns=df_acteur_from_sql.columns
)

parent_ids = df_revisionacteur_from_sql['parent_id'].dropna().unique().tolist()
parent_id_list = [f"'{i}'" for i in parent_ids]
if parent_ids:
    query_parentacteur = f"SELECT * FROM qfdmo_revisionacteur WHERE identifiant_unique IN ({','.join(parent_id_list)});"
    df_parentacteur_from_sql = pd.read_sql_query(query_parentacteur, engine)
    df_parentacteur_from_sql['source_id'] = 0
    df_parentacteur_from_sql['is_parent'] = True

    # récupération des enfants des parents pour les inclure dans les clusters
    query_enfantacteur = f"SELECT * FROM qfdmo_revisionacteur WHERE parent_id IN ({','.join(parent_id_list)});"
    df_enfantacteur_from_sql = pd.read_sql_query(query_enfantacteur, engine)
    df_enfantacteur_from_sql['is_parent'] = False



In [168]:
# Ajout du cluster_id dans les acteurs
df_acteur_from_sql = df_acteur_from_sql.merge(df_clusters, on='identifiant_unique', how='left')

# Ajout du cluster_id dans les parents en inspectant si le cluster des enfant dans df_acteur_from_sql
df_parentacteur_from_sql["cluster_id"] = df_parentacteur_from_sql['identifiant_unique'].apply(
    lambda x: df_acteur_from_sql[df_acteur_from_sql['parent_id'] == x]['cluster_id'].values[0])

# ajout du cluster_id dans les enfants en inspectant le cluster_id de leur parent
df_enfantacteur_from_sql["cluster_id"] = df_enfantacteur_from_sql['parent_id'].apply(
    lambda identifiant_unique: df_parentacteur_from_sql[df_parentacteur_from_sql['identifiant_unique'] == identifiant_unique]['cluster_id'].values[0]
)

## Compilation des acteurs chapeau

- Regroupement des acteurs par cluster
- Choix de la valeur à appliquer pour chacun des champs

In [169]:
df_acteur_merged = pd.concat([df_acteur_from_sql, df_parentacteur_from_sql, df_enfantacteur_from_sql])
df_acteur_merged.drop_duplicates(subset=['identifiant_unique'], inplace=True)
grouped_df = df_acteur_merged.groupby('cluster_id').apply(lambda x: x.to_dict("records") if not x.empty else [])

# Ordonner chaque groupe par ordre de confiance selon la liste trusted_sources : 
# D'abord la source ALIAPUR, puis COREPILE, puis les autres sources
grouped_df = grouped_df.apply( 
    lambda x: (
        sorted(x, key=lambda y: 
               trusted_source_ids.index(y['source_id']) 
               if y['source_id'] in trusted_source_ids
               else len(trusted_sources)
        )
    )
)

# print(grouped_df.head())
acteurchapeau_par_cluster = {}
for cluster_id, group in grouped_df.items():
    # Pour chacune des colonnes columns
    # On prend dans l'ordre la valeur de la première ligne (group) si elle est non nulle
    # Puis la valeur de la suivante si elle est non nulle
    # etc
    combined_row = {}
    for column in df_acteur_merged.columns:
        for record in group:
            if pd.isnull(record[column]) or pd.isna(record[column]) or record[column] == '':
                break
            combined_row[column] = record[column]

    combined_row['identifiant_unique'] = str(uuid.uuid4())
    acteurchapeau_par_cluster[cluster_id] = combined_row

#acteurs_chapeau_par_cluster = pd.DataFrame(acteurs_chapeau_par_cluster)
acteurs_chapeau = acteurchapeau_par_cluster.values()
df_acteurchapeau = pd.DataFrame(acteurs_chapeau)


/var/folders/2p/9gsx3n6n4kn4qn8lzv1v8llr0000gn/T/ipykernel_7843/2682940684.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_acteur_merged = pd.concat([df_acteur_from_sql, df_parentacteur_from_sql, df_enfantacteur_from_sql])


## Création des acteurs chapeau en base de données

In [170]:
# Suppression des colonnes qui ne sont pas en base de données
cleaned_acteurs_chapeau_df = df_acteurchapeau.drop(columns=['is_parent', 'cluster_id'])
# Write to the database
cleaned_acteurs_chapeau_df.to_sql(
    'qfdmo_revisionacteur',
    engine,
    if_exists="append",
    index=False,
    method="multi",
    chunksize=1000,
)

2649

## Créaction et mise à jour des RevisionActeurs

In [177]:
# retries = 3
from tenacity import retry, wait_fixed, stop_after_attempt, retry_if_exception_type
@retry(
    wait=wait_fixed(5),
    stop=stop_after_attempt(3),
    retry=retry_if_exception_type(Exception)
)
def get_or_create_revisionacteur(identifiant_unique):
    response = requests.get(
        f"{INTERFACE_HOST}/qfdmo/getorcreate_revisionacteur/{identifiant_unique}"
    )
    if response.status_code >= 400:
        raise Exception(f"Error while getting or creating revisionacteur for {identifiant_unique}")
    return response.json()


for cluster_id, group in grouped_df.items():
    for record in group:
        if record['is_parent']:
            continue
        # Creation du revision acteur pour chaque acteur à dédupliquer
        # TODO : ajouter un retries
        response = requests.get(
            f"{INTERFACE_HOST}/qfdmo/getorcreate_revisionacteur/"
            f"{record['identifiant_unique']}"
        )
        # Mise à jour du parent_id avec l'identifiant_unique de l'acteur chapeau
        parent_id = acteurchapeau_par_cluster[cluster_id]['identifiant_unique']
        query_acteur = f"""
            UPDATE qfdmo_revisionacteur
            SET parent_id = '{parent_id}'
            WHERE identifiant_unique = '{record['identifiant_unique']}';
        """
        engine.execute(query_acteur)



In [178]:
# Remove old parent
if parent_ids:
    query_delete_old_parentacteur = f"""DELETE FROM qfdmo_revisionacteur WHERE identifiant_unique IN ({','.join(parent_id_list)});"""
    engine.execute(query_delete_old_parentacteur)

# Outils pour tester

Supprimer tous les acteurs chapeau qui viennent d'être créés.

⚠️ Ne pas utiliser hors de l'environnement de développement

In [103]:
# Tooling temporaire


# supprimer les acteurs chapeau identifié pas cleaned_acteurs_chapeau_df['identifiant_unique']
identifiant_uniques = cleaned_acteurs_chapeau_df['identifiant_unique'].tolist()
query_acteur = f"UPDATE qfdmo_revisionacteur SET parent_id = NULL WHERE parent_id IN ({', '.join(f"'{id}'" for id in identifiant_uniques)});"
engine.execute(query_acteur)
query_acteur = f"DELETE FROM qfdmo_revisionacteur WHERE identifiant_unique IN ({', '.join(f"'{id}'" for id in identifiant_uniques)});"
engine.execute(query_acteur)


## TODO

 - [x] Récupérer les données des acteurs + revisionacteurs à partir de la base de données 
 - [x] Pensez à regarder si un des acteurs à dédupliquer a déjà un acteur chapeau 
